In [148]:
import json
import psycopg2
import requests
import time

In [149]:
#Define Variables

# Replace domain_name with your zendesk domain name
zd_addr="<domain_name>.zendesk.com"
zd_addr="yugabyte.zendesk.com"

zd_user_token="XXXXXXXXXXXXXXXX"
zd_user_token="XtqGMUzMfHZP9vBv0J4CWWM1989zvQfVeXyKkgqi"

user='kmaheshwari@yugabyte.com'  + '/token'
# Format for user name required by zendesk

count=1
#How many cases information to pull from zendesk

In [150]:
#Make a connection to the database

## Repalce your API Token
headers = {
    'Content-Type': "application/json",
    'TOKEN': zd_user_token
    }

base_url="https://"+zd_addr+"/api/v2/tickets.json" + "?per_page=1"

In [151]:
yb_db_addr="127.0.0.1"
yb_stats_db="zendesk_db"
yb_db_port="5433"
yb_db_user="yugabyte"
yb_db_password=""
yb_db_ssl_path=""

In [152]:
try:
# Connect to your postgres DB
    conn = psycopg2.connect(dbname=yb_stats_db,host=yb_db_addr,port=yb_db_port,user=yb_db_user)

# Open a cursor to perform database operations
    cur = conn.cursor()
except Exception as err:
    print ("Oops! An exception has occured:", error)
    print ("Exception TYPE:", type(error))

In [ ]:
insert_sql="insert into ticket1(data) values (%s)"
url=base_url
while True:
    response = requests.get(url, auth=(user, zd_user_token))
    data= response.json()
    dst=json.dumps(data)
    cur.execute(insert_sql,(dst,))
    conn.commit()
    url=data['next_page']
    print(url)
    if (data['next_page'] is None):
        print("Next page is null")
        print(data['next_page'])
        break

    #How many cases information to pull from zendesk. By default, setting the counter to 2 
    count=count+1
    if count > 2:
        break
    
    # Sleep 1 second to avoid hitting API Rate throttling from zendesk
    time.sleep(1)

https://yugabyte.zendesk.com/api/v2/tickets.json?page=2&per_page=1


In [ ]:
# Retrieve data from the table
select_sql="select data -> 'tickets' -> 0 -> 'id' as Case_no, data -> 'tickets' -> 0 -> 'url' AS Current_URL, data -> 'tickets' -> 0 -> 'via' -> 'channel' AS channel, data -> 'next_page' AS \"next page\" from ticket1"
cur.execute(select_sql)
result = cur.fetchall()

for i in range(len(result)):
    print(result[i])

In [ ]:
# Total number of rows in table
select_sql="select count(*) from ticket1"
cur.execute(select_sql)
result = cur.fetchone()
print("Total number of rows: " + str(result[0]))

In [147]:
cur.close()